Prueba de cálculo de salario promedio general  y desagregado por sexo en la Ciudad de México

Serie longitudinal: últimos trimestres de 2021-2024

In [3]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, writexl)

Descargar y leer los archivos csv

In [4]:
dir_datos <- 'enoe_datos'
if (!dir.exists(dir_datos)) {
    dir.create(dir_datos)
}

In [5]:
descargar_enoe_csv <- function() {
    options(timeout = 90000)

    descargar_extraer <- function(url, destino) {
        temp <- tempfile()
        download.file(url, temp, mode = 'wb')
        unzip(temp, exdir = destino)
        unlink(temp)

    }
    
    #iterar los años de descarga    
    for(i in 21:22) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20', i, '_trim4_csv.zip')
        descargar_extraer(url, dir_datos)
    }

    for(i in 23:24) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20', i, '_trim4_csv.zip')
        descargar_extraer(url, dir_datos)
    }
}

In [6]:
descargar_enoe_csv()

In [7]:
archivos <- list.files(path = dir_datos, pattern = 'sdem.*\\.csv$', full.names = TRUE, ignore.case = TRUE)

datos <- archivos %>%
lapply(read_csv, show_col_types = FALSE)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [8]:
tablas <- list()  # Crear lista vacía para almacenar los resultados

for (i in seq_along(datos)) {
  tablas[[i]] <- datos[[i]] %>%
    clean_names() %>%
    mutate(ano = as.numeric(str_extract(archivos[i], '\\d{2}(?=\\.csv$)')))
}

sdem <- bind_rows(tablas)


Diseño muestral

In [9]:
#Escenario complejo
enoe_svy <- as_survey_design(
    .data = sdem,
    ids = upm,
    strata = est_d_tri,
    weights = fac_tri,
    nest = TRUE 
) 

Cálculo de salario promedio

In [10]:
#Población general
gral_svy <- enoe_svy %>%
filter(r_def ==00, (c_res == 1 | c_res == 3),
         eda >= 15  & eda <= 98) %>%
filter(ent == 9, clase2 == 1) #Filtrar por población ocupada en la Ciudad de México

In [11]:
#Salario promedio general
salario_general <- gral_svy %>%
group_by(ano) %>%
summarise(sal_prom_gral = survey_mean(ingocup, na.rm = TRUE)) %>%
select(-sal_prom_gral_se)

In [12]:
#Salario promedio por sexo
salario_sex <- gral_svy %>%
group_by(ano, sex) %>%
summarise(sal_prom = survey_mean(ingocup, na.rm = TRUE)) %>%
select(-sal_prom_se) %>%
pivot_wider(names_from = sex, values_from = sal_prom, names_prefix = "sal_prom_")

In [13]:
salario_general <- salario_general %>%
right_join(salario_sex, by = 'ano') %>%
select(ano, sal_prom_gral, sal_prom_1, sal_prom_2) %>%
rename(sal_prom_hombres = sal_prom_1, sal_prom_mujeres = sal_prom_2)